# Coursework 1: Finite Fields
# Group 8

[√] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

## 1.1 Integers (30%)

### 1.1.1 Check a prime number

Write a function `isprime` which checks whether a positive integer is a prime or not. Test your function using at least five different numbers (some primes some not) less than 100. 

In [1]:
# Levi's implementation
function isprime(n)
    if n < 2
        return false    
    elseif n == 2
        return true
    # Check all even numbers
    elseif n % 2 == 0
        return false
    end

    # Consider all odd numbers
    for i in collect(3:2:√n) # only need to check i < √n
        if n % i == 0 
            return false
        end
    end

    # if no divisors were found
    true  
end


testInts = 0:99

# Print all primes from 0 to 99
testInts .|> (x) -> print(isprime(x) ? "$x  " : "")
nothing

2  3  5  7  11  13  17  19  23  29  31  37  41  43  47  53  59  61  67  71  73  79  83  89  97  

### 1.1.2 Extended Euclidean algorithm for integers

1. Get familiar with the function `gcdx` that comes with Julia. 
2. Mimic the input and output style of `gcdx` and write your own function `my_gcdx` for the same functionality.
3. Test your function using at least 5 different instances. 

In [2]:
println(gcdx(3,12))
println(gcdx(12,3))
println(gcdx(46,72))
println(gcdx(46,71))
println(gcdx(13,5))

(3, 1, 0)
(3, 0, 1)
(2, 11, -7)
(1, 17, -11)
(1, 2, -5)


### Derivation

In order to implement `my_gcdx`, the first 3 lines of the proof for Bézout's Identity were analysed and a pattern spotted.   


(i=1)&nbsp;&nbsp;&nbsp;&nbsp; $ r_n = -q_nr_{n-1} + r_{n-2} $ 

(i=2)&nbsp;&nbsp;&nbsp;&nbsp;$ r_n = -q_n(-q_{n-1}r_{n-2} + r_{n-3}) + r_{n-2} = (1+q_nq_{n-1})r_{n-2} - q_nr_{n-3} $

(i=3)&nbsp;&nbsp;&nbsp;&nbsp;$ r_n = (1+q_nq_{n-1})(-q_{n-2}r_{n-3} + r_{n-4}) - q_nr_{n-3} = -(q_n + q_{n-2} + q_n q_{n-1}   q_{n-2})r_{n-3} + (1+ q_nq_{n-1})r_{n-4} $

Let's call the coefficient of the $r_{n-1-i}$ term x_i and the coefficient of the $r_{n-i}$ term y_i. It was noticed that:

$x_{i+1} = y_i$ and $y_{i+1} = x_i - y_i  q_{n-i}$. 

This can be easily verified by simply starting with $i=1$ as first equation above and plugging in $x_2 = y_1 = -q_n$ and $y_2 = x_1 - y_1  q_{n-1} = 1 - (-q_n)(q_{n-1})$ as shown in the second line of the proof. This can be performed for every line of the Euclidean Algorithm (backwards). This is implemented below as a recursive function where first the function is called recursively and calculates gcd then when it unwinds it performs this iterative equation. 

In order to find the values of $x_0$ and $y_0$ in the base case where the remainder is 0, this relationship can be performed on $i=0$:

$ x_1 = 1 $

$ y_1 = -q_n $

$ x_1 = y_0 \rightarrow y_0 = 1$

$ y_1 = x_0 - y_0 \cdot q_n \rightarrow -q_n = x_0 -1 \cdot q_n \rightarrow x_0 = 0 $

This gives us x=0 and y=1 for our base case below





In [3]:
function my_gcdx(a, b)
    remainder = a % b
    quotient = div(a, b)

    if remainder == 0 
        # when the remainder is 0, r_n = 0*a + 1*b
        return (gcd = b, x = 0, y = 1)
    else
        # calc prev level
        prev = my_gcdx(b, remainder)
        # propagate gcd and multiply x and y using formula derived above
        return (gcd = prev.gcd, x= prev.y, y = (prev.x - prev.y * quotient))
    end
end
# gcd = x*a + y*b

println(my_gcdx(3,12))
println(my_gcdx(12,3))
println(my_gcdx(46,72))
println(my_gcdx(46,71))
println(my_gcdx(13,5))

(gcd = 3, x = 1, y = 0)
(gcd = 3, x = 0, y = 1)
(gcd = 2, x = 11, y = -7)
(gcd = 1, x = 17, y = -11)
(gcd = 1, x = 2, y = -5)


### 1.1.3 Modular arithmetic 

Let $p=811$, $x=3$, and $n=789$. Compute $x^n \equiv a ~\text{mod}~p$. Show the necessary steps for the computations. 

In [4]:
# start doing x^2..n check after each mult if greater than p modulate it and then continue

function modExponent(x,n,p)
  sum = x
  while n!=1
    # mult by x again, apply modulation to the result
    sum = (sum * x) % p
    n = n-1
  end
  return sum
end

modExponent(3,789,811)

188

## 1.2 Polynomials (50%)

Let $p$ be a prime number. Consider the polynomial ring $\mathbb{F}_p[x]$. 

### 1.2.1 Polynomial division

Design and write a function `polydiv' which calculates the quotient $q(x)$ and the remainder $r(x)$ from $f(x) / g(x)$ so that 
1. $f(x) = q(x) g(x) + r(x)$ where $\text{deg}(r(x)) < \text{deg}(g(x))$; 
2. Clear documentation to allow other people to know how to input $f(x)$ and $g(x)$. 
3. Run 5 different tests with simple cases. For example, $p=2$ and $p=3$, $\text{deg}(f) = 3$ and $\text{deg}(f) = 4$. 

### Explanation

In this section, the core function is 'polydiv'. We write a few functions to support 'polydiv' and do not use any external packages.

Our idea is to use matrixs to represent polynomials, e.g. [1 2 3] -> 1 + 2*x + 3*x^2. 

In 'polydiv', we let the last element of q equals to the last coefficient of f(x) ÷ the last coefficient of g(x). (e.g. for f(x) = [1 2 3 4], g(x) = [1 2], q_last = 4 ÷ 2). Then, let f(x) = f(x) - g(x)*q

Continue the above algorithm, we finally get all the elements of q(x), and r(x) is the latest f(x).

If coefficient of f(x) ÷ coefficient of g(x) is not an integer, we also designed a function 'fractionmod' to get the modulation of this fraction.


In [5]:
# Used to determine whether the input is a prime number 
# Return 'true' if it is a prime number, return 'false' if not
function isprime(n) 
        # exclude 1
        if n == 1 
            return false     
        # exclude 2
        elseif n == 2
            return true 
        end
        # Start to exam from 3,determine whether it can be divisible by numbers form 2-(√n+1)
        for i = 2:floor(sqrt(n)+1) 
            if (n % i) == 0
                return  false
            end
        end        
        # None of the above, hence n is a prime number
        return true 
end

# Determine the degree of polynomial A. On the right is the coefficient of the largest term
# 3x^2+2x+1 then input A = [1 2 3]
function getdeg(A)
    n = length(A)

    for i in 1:length(A)
        if A[n] == 0 # Determine whether the last bit is 0, if yes, reduce 1 to the length of A
            n -= 1; # Judge the next one in the next loop
            if n == 0 # if all of A equal to 0, then return degree 0
                return 0
            end
        else
            return n-1 # degree starts from 0, and the number of items starts from 1, so return n-1
        end
    end
end

# Fraction modulation 
function fractionmod(molecular,denominator,p) # input the molecular and denominator of the fraction, then get the mod
    n=0 
    judgement = true #make judgement whether jump out the loop
    while judgement
        n += 1
        judgement = (mod(molecular,p) != mod(denominator*n,p)) 
        #Judge how many n can make the polynomial hold, they molecular%p equal to denominator module p, return n
    end
    return n
end

# get the modulus of each number in a matrix
function arraymod(array,p) #input the matrix
    for i = 1:length(array)  
        array[i]=mod(array[i],p) # Modulo each item in the matrix
    end
    return array
end

# show the matrix in the form of polynomial, e.g. [1 2 3] -> 1 + 2*x + 3*x^2
function show_poly(ax)
    L = length(ax)
    if getdeg(ax) == 0 # if degree of ax is 0, output its first element
        print(ax[1])
        print("\n")
    else # if ax has more than 1 element
        flag = 0 # this flag is used to make sure the sign " + " is correctly demonstrated
        for i = 1:L
            b = ax[i]
            if b != 0 # if b = 0, do not output, print if b != 0
                if flag == 1 # if there is a output before this output, output " + ". This is used to avoid this kind of output: " + 2*x + 3*x^2"
                    print(" + ")
                end
                if i == 1 # output the element that x has 0 degree, hence only output b, rather than b*x^0
                    print("$b")
                elseif i == 2 # output the element that x has 1 degree, hence only output bx, rather than b*x^1
                    if b == 1 # check whether b = 1, if true, only output x, rather than 1*x
                        print("x")
                    else
                        print("$b*x") # output bx
                    end
                else
                    if b == 1
                        print("x^$(i-1)") # check whether b = 1, if true, only output x^(i-1), rather than 1*x^(i-1)
                    else
                        print("$b*x^$(i-1)") # output b*x^(i-1)
                    end
                end
                flag = 1
            end
        end
        print("\n") # add a final linefeed to the output
    end
end

# Polynomial division, Find the remainder and quotient of fx/gx in p-domain
function polydiv(fx,gx,p) 
    if isprime(p) # decide it's prime or not, if not, then return error
        fx = arraymod(fx,p) # Take the modulus of fx to ensure that coefficients of fx is in Fp
        gx = arraymod(gx,p) # Take the modulus of gx to ensure that coefficients of gx is in Fp
        deg_fx = getdeg(fx) # get the degree of fx and gx
        deg_gx = getdeg(gx)
        fx = fx[1,1:deg_fx+1]' # Discard the extra 0 in fx, e.g. [1 2 1 0 0] -> [1 2 1]
        gx = gx[1,1:deg_gx+1]' # Discard the extra 0 in gx

        if gx == [0]'
            println("Warning, cannot divide 0!")
            return (rem=[],quo=[])
        elseif deg_gx > deg_fx # If the order of gx is higher than fx, the remainder is gx, the quotient is 0
            rx = convert(Matrix{Int64},fx)
            qx = [0]
            return (rem=rx,quo=qx)
        else
            deg_diff = deg_fx - deg_gx # degree different between fx and gx
            qx = zeros(1,deg_diff+1) # Initialize qx and rx
            rx = zeros(Int64,1,deg_gx+1)

            for i in 0:deg_diff
                # Add 0 to the gx matrix to make gx and fx the same length
                # With the change of degree quotient, the position of gx is different each time
                zero_highdeg = zeros(Int64,1,i) 
                zero_lowdeg = zeros(Int64,1,deg_diff-i)
                gx_addzero = [zero_lowdeg gx zero_highdeg]

                # Calculate the quotient of each step
                # eg: Divide the highest order coefficient of fx by the highest order coefficient of gx for the first time
                qx[deg_diff+1-i] = fx[deg_fx+1-i]/gx[deg_gx+1]

                # Modulo qx
                if !isinteger(qx[deg_diff+1-i]) # Fraction modulo if qx is not a integer
                    qx[deg_diff+1-i] = fractionmod(fx[deg_fx+1-i],gx[deg_gx+1],p)
                else # integer mod
                    qx[deg_diff+1-i] = mod(qx[deg_diff+1-i],p) 
                end

                # Calculate the remainder, the remainder is the next divider
                fx = fx-gx_addzero*qx[deg_diff+1-i]
            end

            # Remove the 0 in rx and gx, and convert them into Int
            rx = convert(Matrix{Int64},arraymod(fx,p)) # the newest fx is the remainder, do modulation
            rx = rx[1,1:getdeg(rx)+1]' # Discard extra 0s
            qx = convert(Matrix{Int64},arraymod(qx,p))
            qx = qx[1,1:getdeg(qx)+1]' # Discard extra 0s

            return (rem=rx,quo=qx)
        end
    else
        return "p is not a prime number. Please change the value of p." # p is not a prime
    end
end

function show_result_1(p,fx,gx,result)
    print("In the polynomial ring F$p[x]\n")
    print("Input fx:  ")
    show_poly(fx)
    print("Input gx:  ")
    show_poly(gx)
    print("The quotient is:  ")
    show_poly(result.quo)
    print("The Remainder is:  ")
    show_poly(result.rem)
    print("\n")
end

# The right hand side of the input matrix has the highest degree, e.g. [1 2 3] -> 1 + 2*x + 3*x^2
fx = [1 2 2 3 1]
gx = [1 2 4]
p = 3
result = polydiv(fx,gx,p)
show_result_1(p,fx,gx,result)

fx = [2 3 5 4 1]
gx = [3 3 2]
p = 5
result = polydiv(fx,gx,p)
show_result_1(p,fx,gx,result)

fx = [1 2 4]
gx = [2 3]
p = 5
result = polydiv(fx,gx,p)
show_result_1(p,fx,gx,result)

fx = [3 2 0 1 1]
gx = [1 0 1]
p = 3
result = polydiv(fx,gx,p)
show_result_1(p,fx,gx,result)

fx = [1 0 4 0 3]
gx = [4 1 4]
p = 5
result = polydiv(fx,gx,p)
show_result_1(p,fx,gx,result)


In the polynomial ring F3[x]
Input fx:  1 + 2*x + 2*x^2 + x^4
Input gx:  1 + 2*x + x^2
The quotient is:  2 + x + x^2
The Remainder is:  2

In the polynomial ring F5[x]
Input fx:  2 + 3*x + 4*x^3 + x^4
Input gx:  3 + 3*x + 2*x^2
The quotient is:  3 + 3*x^2
The Remainder is:  3 + 4*x

In the polynomial ring F5[x]
Input fx:  1 + 2*x + 4*x^2
Input gx:  2 + 3*x
The quotient is:  2 + 3*x
The Remainder is:  2

In the polynomial ring F3[x]
Input fx:  2*x + x^3 + x^4
Input gx:  1 + x^2
The quotient is:  2 + x + x^2
The Remainder is:  1 + x

In the polynomial ring F5[x]
Input fx:  1 + 4*x^2 + 3*x^4
Input gx:  4 + x + 4*x^2
The quotient is:  1 + 2*x + 2*x^2
The Remainder is:  2 + x



### 1.2.2 Extended Euclidean algorithm for polynomials

Design and write a function `gcdx_poly` to compute the GCD of $f(x)$ and $g(x)$, and their Bezout coefficients. Run 5 different tests to demonstrate the correctness of your function.

### Explanation

The first 6 functions are the same as 1.2.1, please go to line 151 directly if you have read 1.2.1.

We first wirte a function for matrix convolution, which can be regarded as the multiplication of polynomials. This function can support 'gcdx_poly'.

Then it comes to 'gcdx_poly'. This function uses the idea of iteration. Let $a = f(x), b = g(x)$. It first calculate $a/b$ and get $q_1, r_1$. Then calculate $b/r_1$ and get $q_2, r_2$; calculate $r_1/r_2$ and get $q_3, r_3$, ..., untill $r_n = 0$.

Therefore, GCD is $r_{n-1}$

$0 = r_n = r_{n-2}-q_n*r_{n-1}$

$r_{n-1} = r_{n-3}-q_{n-1}*r_{n-2}$

$r_{n-2} = r_{n-4}-q_{n-2}*r_{n-3}$

$r_{n-3} = r_{n-5}-q_{n-3}*r_{n-4}$

...

$r_3 = r_1-q_3*r_2$

$r_2 = b-q_2*r_1$

$r_1 = a-q_1*b$

These can help us find GCD = $x*a + y*b$

In the last iteration (where $r_n = 0$), let $x_n = 0, y_n = 1$, then in the previous iteration, let $x_{n-1} = y_n, y_{n-1} = x_n - y_n*q_{n-1}$

Continue this we get, $x_1 = y_2, y_1 = x_2 - y_2*q_1$, it can be easily prove that GCD = $x_1*a + y_1*b$, where GCD is $r_{n-1}$


Please directly go to line 151 if you have read 1.2.1, the front functions are the same. In order to see the line number, you can click the left side (just bellow the run button) of the code block and type"L".

In [6]:
# Used to determine whether the input is a prime number 
# Return 'true' if it is a prime number, return 'false' if not
function isprime(n) 
        # exclude 1
        if n == 1 
            return false     
        # exclude 2
        elseif n == 2
            return true 
        end
        # Start to exam from 3,determine whether it can be divisible by numbers form 2-(√n+1)
        for i = 2:floor(sqrt(n)+1) 
            if (n % i) == 0
                return  false
            end
        end        
        # None of the above, hence n is a prime number
        return true 
end

# Determine the degree of polynomial A. On the right is the coefficient of the largest term
# 3x^2+2x+1 then input A = [1 2 3]
function getdeg(A)
    n = length(A)

    for i in 1:length(A)
        if A[n] == 0 # Determine whether the last bit is 0, if yes, reduce 1 to the length of A
            n -= 1; # Judge the next one in the next loop
            if n == 0 # if all of A equal to 0, then return degree 0
                return 0
            end
        else
            return n-1 # degree starts from 0, and the number of items starts from 1, so return n-1
        end
    end
end

# Fraction modulation 
function fractionmod(molecular,denominator,p) # input the molecular and denominator of the fraction, then get the mod
    n=0 
    judgement = true #make judgement whether jump out the loop
    while judgement
        n += 1
        judgement = (mod(molecular,p) != mod(denominator*n,p)) 
        #Judge how many n can make the polynomial hold, they molecular%p equal to denominator module p, return n
    end
    return n
end

# get the modulus of each number in a matrix
function arraymod(array,p) #input the matrix
    for i = 1:length(array)  
        array[i]=mod(array[i],p) # Modulo each item in the matrix
    end
    return array
end

# show the matrix in the form of polynomial, e.g. [1 2 3] -> 1 + 2*x + 3*x^2
function show_poly(ax)
    L = length(ax)
    if getdeg(ax) == 0 # if degree of ax is 0, output its first element
        print(ax[1])
        print("\n")
    else # if ax has more than 1 element
        flag = 0 # this flag is used to make sure the sign " + " is correctly demonstrated
        for i = 1:L
            b = ax[i]
            if b != 0 # if b = 0, do not output, print if b != 0
                if flag == 1 # if there is a output before this output, output " + ". This is used to avoid this kind of output: " + 2*x + 3*x^2"
                    print(" + ")
                end
                if i == 1 # output the element that x has 0 degree, hence only output b, rather than b*x^0
                    print("$b")
                elseif i == 2 # output the element that x has 1 degree, hence only output bx, rather than b*x^1
                    if b == 1 # check whether b = 1, if true, only output x, rather than 1*x
                        print("x")
                    else
                        print("$b*x") # output bx
                    end
                else
                    if b == 1
                        print("x^$(i-1)") # check whether b = 1, if true, only output x^(i-1), rather than 1*x^(i-1)
                    else
                        print("$b*x^$(i-1)") # output b*x^(i-1)
                    end
                end
                flag = 1
            end
        end
        print("\n") # add a final linefeed to the output
    end
end

# Polynomial division, Find the remainder and quotient of fx/gx in p-domain
function polydiv(fx,gx,p) 
    if isprime(p) # decide it's prime or not, if not, then return error
        fx = arraymod(fx,p) # Take the modulus of fx to ensure that coefficients of fx is in Fp
        gx = arraymod(gx,p) # Take the modulus of gx to ensure that coefficients of gx is in Fp
        deg_fx = getdeg(fx) # get the degree of fx and gx
        deg_gx = getdeg(gx)
        fx = fx[1,1:deg_fx+1]' # Discard the extra 0 in fx, e.g. [1 2 1 0 0] -> [1 2 1]
        gx = gx[1,1:deg_gx+1]' # Discard the extra 0 in gx

        if gx == [0]'
            println("Warning, cannot divide 0!")
            return (rem=[],quo=[])
        elseif deg_gx > deg_fx # If the order of gx is higher than fx, the remainder is gx, the quotient is 0
            rx = convert(Matrix{Int64},fx)
            qx = [0]
            return (rem=rx,quo=qx)
        else
            deg_diff = deg_fx - deg_gx # degree different between fx and gx
            qx = zeros(1,deg_diff+1) # Initialize qx and rx
            rx = zeros(Int64,1,deg_gx+1)

            for i in 0:deg_diff
                # Add 0 to the gx matrix to make gx and fx the same length
                # With the change of degree quotient, the position of gx is different each time
                zero_highdeg = zeros(Int64,1,i) 
                zero_lowdeg = zeros(Int64,1,deg_diff-i)
                gx_addzero = [zero_lowdeg gx zero_highdeg]

                # Calculate the quotient of each step
                # eg: Divide the highest order coefficient of fx by the highest order coefficient of gx for the first time
                qx[deg_diff+1-i] = fx[deg_fx+1-i]/gx[deg_gx+1]

                # Modulo qx
                if !isinteger(qx[deg_diff+1-i]) # Fraction modulo if qx is not a integer
                    qx[deg_diff+1-i] = fractionmod(fx[deg_fx+1-i],gx[deg_gx+1],p)
                else # integer mod
                    qx[deg_diff+1-i] = mod(qx[deg_diff+1-i],p) 
                end

                # Calculate the remainder, the remainder is the next divider
                fx = fx-gx_addzero*qx[deg_diff+1-i]
            end

            # Remove the 0 in rx and gx, and convert them into Int
            rx = convert(Matrix{Int64},arraymod(fx,p)) # the newest fx is the remainder, do modulation
            rx = rx[1,1:getdeg(rx)+1]' # Discard extra 0s
            qx = convert(Matrix{Int64},arraymod(qx,p))
            qx = qx[1,1:getdeg(qx)+1]' # Discard extra 0s

            return (rem=rx,quo=qx)
        end
    else
        return "p is not a prime number. Please change the value of p." # p is not a prime
    end
end

function matrix_conv(a, b) # our own convolution function
    M = length(a)          # the multiplication of polynomials is actually the convolution
    N = length(b)
    c = zeros(Int64,1,M+N-1)
    for i = 1:M
        for j = 1:N
            c[i+j-1] += a[i]*b[j] # find each element
        end
    end
    return c
end

# Find the GCD and bezout's identity (x,y) of fx and gx
function gcdx_poly(fx,gx,p)
    result = polydiv(fx,gx,p) # Polynomial division, Find the remainder and quotient of fx/gx in p-domain
    remainder = result.rem
    quotient = result.quo
    if getdeg(remainder) == 0 && remainder[1] == 0 # when the remainder is 0, gcd = 0*fx + 1*gx
        gcd = convert(Matrix{Int64},arraymod(gx,p))
        gcd = gcd[1,1:getdeg(gcd)+1]' # Discard extra 0s

        L_gcd = length(gcd)
        y = ones(Int64,1,1)
        if gcd[L_gcd] !=1 # make the coefficient of the highest degree x equals to 1
            y[1] = fractionmod(y[1],gcd[L_gcd],p)
            for j = 1:L_gcd
                gcd[j] = fractionmod(gcd[j],gcd[L_gcd],p)
            end
        end
        return (gcd, x = zeros(Int64,1,1), y)
    else
        prev = gcdx_poly(gx,remainder,p) # iteration, untill the remainder is 0
        a = matrix_conv(prev.y, quotient)  # calculate prev.y*quotient (generally, quotient is not the same for every iteration)

        # because the length of a and prev.x are not the same, this is used to calculate (prev.x - a), which is actually (prev.x - prev.y*quotient)
        if length(a) > length(prev.x)
            y = -a
            for i = 1:length(prev.x)
                y[i] += prev.x[i]
            end
        else
            y = prev.x
            for i = 1:length(a)
                y[i] -= a[i]
            end
        end

        # gcd is always the same, x = prev.y, y = prev.x - prev.y*quotient
        y = convert(Matrix{Int64},arraymod(y,p))
        y = y[1,1:getdeg(y)+1]'
        return (gcd = prev.gcd, x= prev.y, y)
    end
end

function show_result_2(p,fx,gx,result)
    print("In the polynomial ring F$p[x]\n")
    print("Input fx:  ")
    show_poly(fx)
    print("Input gx:  ")
    show_poly(gx)
    print("GCD is:  ")
    show_poly(result.gcd)
    print("x is:  ")
    show_poly(result.x)
    print("y is:  ")
    show_poly(result.y)
    print("(GCD = x * fx + y * gx)")
    print("\n\n")
end

# The right hand side of the input matrix has the highest degree, e.g. [1 2 3] -> 1 + 2*x + 3*x^2
fx = [2 3 0 4 1]
gx = [3 3 2]
p = 5
result = gcdx_poly(fx,gx,p)
show_result_2(p,fx,gx,result)


In the polynomial ring F5[x]
Input fx:  2 + 3*x + 4*x^3 + x^4
Input gx:  3 + 3*x + 2*x^2
GCD is:  2 + x
x is:  4
y is:  3 + 3*x^2
(GCD = x * fx + y * gx)



### 1.2.3 Irreducible polynomial

Design and write a function `is_irreducible` to check whether $f(x) \in \mathbb{F}_p[x]$ is irreducible or not. Run 5 tests to demonstrate the correctness of your function. 

### **Explanation**
If $f(x)$ is reducible, that means we can find $a(x), b(x)$, such that $f(x) = a(x)*b(x)$. Degree of $a(x)$ and $b(x)$ should be smaller than $f(x)$.

Since mathematicians have not discovered the necessary condition of wherther a function is irreducible, and some famous criterions, such as Eisenstein's irreducibility criterion, are just sufficient conditions, we have to use exhaustive search.

 If we list all the possibilities of $a(x)$. Check whether $a(x)$ can be divisible by $f(x)$, we will know whether $f(x)$ is irreducible.

For $a(x)$, we only need to condider degree equal and smaller than $fld(\frac{degree\, of\, f(x)}{2})$. For example, if we have $f(x)$ with degree 9. We consider 4 as the maximum degree of $a(x)$ (Round down).  

For a polynomial whose deg is n and modulo p, there are $p^n$ possible situations. We express all possibilities in decimal, and then convert them to p carry numbers, which corresponds to an array of polynomials.

For example, for $a(x)$ with maximum degree 1 and mod 3, we have following $3^2$ posibilities (On the right is the highest order coefficient):

$$[0\ 0]=0 \quad  [1\ 0]=1 \quad  [2\ 0]=2$$
$$[0\ 1]=3 \quad  [1\ 1]=4 \quad  [2\ 1]=5$$
$$[0\ 2]=6 \quad  [1\ 2]=7 \quad  [2\ 2]=8$$

Then do the polynomual divide, judge whether the remainder is 0. If the remainder equal to 0, that means the polynomial is irreducible.


Please directly go to line 150 if you have read 1.2.1 and 1.2.2, the front functions are the same. In order to see the line number, you can click the left side (just bellow the run button) of the code block and type"L".

In [7]:
# Determine the degree of polynomial A. On the right is the coefficient of the largest term
# 3x^2+2x+1 then input A = [1 2 3]

function show_poly(ax)
    L = length(ax)
    if getdeg(ax) == 0 # if degree of ax is 0, output its first element
        print(ax[1])
        print("\n")
    else # if ax has more than 1 element
        flag = 0 # this flag is used to make sure the sign " + " is correctly demonstrated
        for i = 1:L
            b = ax[i]
            if b != 0 # if b = 0, do not output, print if b != 0
                if flag == 1 # if there is a output before this output, output " + ". This is used to avoid this kind of output: " + 2*x + 3*x^2"
                    print(" + ")
                end
                if i == 1 # output the element that x has 0 degree, hence only output b, rather than b*x^0
                    print("$b")
                elseif i == 2 # output the element that x has 1 degree, hence only output bx, rather than b*x^1
                    if b == 1 # check whether b = 1, if true, only output x, rather than 1*x
                        print("x")
                    else
                        print("$b*x") # output bx
                    end
                else
                    if b == 1
                        print("x^$(i-1)") # check whether b = 1, if true, only output x^(i-1), rather than 1*x^(i-1)
                    else
                        print("$b*x^$(i-1)") # output b*x^(i-1)
                    end
                end
                flag = 1
            end
        end
        print("\n") # add a final linefeed to the output
    end
end

function getdeg(A)
    n = length(A)

    for i in 1:length(A)
        if A[n] == 0 # Determine whether the last bit is 0, if yes, reduce 1 to the length of A
            n -= 1; # Judge the next one in the next loop
            if n == 0 # if all of A equal to 0, then return it's 0
                return 0
            end
        else
            return n-1 # degree starts from 0, and the number of items starts from 1, so return n-1
        end
    end
end

# Used to determine whether the input is a prime number 
# Return 'true' if it is a prime number, return 'false' if not
function isprime(n) 
        # exclude 1
        if n == 1 
            return false     
        # exclude 2
        elseif n == 2
            return true 
        end     
        # Start to exam from 3,determine whether it can be divisible by numbers form 2-(√n+1)
        for i = 2:floor(sqrt(n)+1) 
            if (n % i) == 0
                return  false
            end
        end        
        # None of the above, hence n is a prime number
        return true 
end

# Fraction modulation 
function fractionmod(molecular,denominator,p) # input the molecular and denominator of the fraction, then get the mod
    n=0 
    judgement = true #make judgement whether jump out the loop
    while judgement
        n += 1
        judgement = (mod(molecular,p) != mod(denominator*n,p)) 
        #Judge how many n can make the polynomial hold, they molecular%p equal to denominator module p, return n
    end
    return n
end

# get the modulus of each number in a matrix
function arraymod(array,p) #input the matrix
    for i = 1:length(array)  
        array[i]=mod(array[i],p) # Modulo each item in the matrix
    end
    return array
end

# Polynomial division, Find the remainder and quotient of fx/gx in p-domain
function polydiv(fx,gx,p) 
    if isprime(p) # decide it's prime or not, if not, then return error
        fx = arraymod(fx,p) # Take the modulus of fx to ensure that coefficients of fx is in Fp
        gx = arraymod(gx,p) # Take the modulus of gx to ensure that coefficients of gx is in Fp
        deg_fx = getdeg(fx) # get the degree of fx and gx
        deg_gx = getdeg(gx)
        fx = fx[1,1:deg_fx+1]' # Discard the extra 0 in fx, e.g. [1 2 1 0 0] -> [1 2 1]
        gx = gx[1,1:deg_gx+1]' # Discard the extra 0 in gx

        if gx == [0]'
            return (rem=[],quo=[])
        elseif deg_gx > deg_fx # If the order of gx is higher than fx, the remainder is gx, the quotient is 0
            rx = convert(Matrix{Int64},fx)
            qx = [0]
            return (rem=rx,quo=qx)
        else
            deg_diff = deg_fx - deg_gx # degree different between fx and gx
            qx = zeros(1,deg_diff+1) # Initialize qx and rx
            rx = zeros(Int64,1,deg_gx+1)

            for i in 0:deg_diff
                # Add 0 to the gx matrix to make gx and fx the same length
                # With the change of degree quotient, the position of gx is different each time
                zero_highdeg = zeros(Int64,1,i) 
                zero_lowdeg = zeros(Int64,1,deg_diff-i)
                gx_addzero = [zero_lowdeg gx zero_highdeg]

                # Calculate the quotient of each step
                # eg: Divide the highest order coefficient of fx by the highest order coefficient of gx for the first time
                qx[deg_diff+1-i] = fx[deg_fx+1-i]/gx[deg_gx+1]

                # Modulo qx
                if !isinteger(qx[deg_diff+1-i]) # Fraction modulo if qx is not a integer
                    qx[deg_diff+1-i] = fractionmod(fx[deg_fx+1-i],gx[deg_gx+1],p)
                else # integer mod
                    qx[deg_diff+1-i] = mod(qx[deg_diff+1-i],p) 
                end

                # Calculate the remainder, the remainder is the next divider
                fx = fx-gx_addzero*qx[deg_diff+1-i]
            end

            # Remove the 0 in rx and gx, and convert them into Int
            rx = convert(Matrix{Int64},arraymod(fx,p)) # the newest fx is the remainder, do modulation
            rx = rx[1,1:getdeg(rx)+1]' # Discard extra 0s
            qx = convert(Matrix{Int64},arraymod(qx,p))
            qx = qx[1,1:getdeg(qx)+1]' # Discard extra 0s

            return (rem=rx,quo=qx)
        end
    else
        return "p is not a prime number. Please change the value of p." # p is not a prime
    end
end

# convert decimal to n-scale, and make the output be a vector with length 'leng'
# Constantly modulo d and p, and string the results together to achieve n-scale conversion
function convertP(d,p,leng) 
    
    Remainder = []
    # calculate the remainder
    while d != 0
        push!(Remainder,mod(d,p))
        d = div(d,p)
    end

    # add zeros to at the end of the vector make it have length 'leng'
    lengdiff = leng - length(Remainder)
    for i = 1:lengdiff
        push!(Remainder,0)
    end

    return Remainder
end

# Determine whether the polynomial is separable
function is_irreducible(fx,p) 
    
    if isprime(p) # if p is not a prime, then return error 
        if getdeg(fx) > 1 # if degree of x smaller than 1, that meand it must irreducible
            judgement = 0
            # Divide fx by every polynomials in side the mod p field  
            # if we get 0, means the polynomial equal to the product of two polynomials, means is not irreducible

            judgedeg = fld(getdeg(fx),2) # We only need to judge half
            leng = judgedeg+1
            for i = p:(p^leng-1)
                ergodic = convertP(i,p,leng) #Gengeate every polynomials
                result = polydiv(fx,ergodic',p) 
           

                if length(result.rem) == 1 && result.rem[1] == 0
                    judgement += 1 
                end 
            end
            
            # If every number in the field brought into the polynomial is not 0, It's irreducible
            if judgement == 0 
                print("fx equals to: ") 
                show_poly(fx)
                print("which is irreducible over mod " ,p, " field")
                print("\n\n")
                return
            else 
                print("fx equals to: ") 
                show_poly(fx)
                print("which is not irreducible over mod " ,p, " field")  
                print("\n\n")
                return
            end
        else
            print("fx equals to: ") 
            show_poly(fx)
            print("which is irreducible over mod " ,p, " field")
            print("\n\n")
            return
        end
    else
        print("p is not a prime number. It's not a filed")
        print("\n\n")

        return 
        
    end
end

fx = [1 0 2 0 1] # On the right is the highest order coefficient of polynomial
p = 3  
is_irreducible(fx,p)

fx = [1 0 2 0 1] 
p = 1 
is_irreducible(fx,p)

fx = [1 0 2 0 1] 
p = 5
is_irreducible(fx,p)

fx = [1 0 1] 
p = 2
is_irreducible(fx,p)

fx = [1 0 1] 
p = 3
is_irreducible(fx,p)

fx = [1 0 1] 
p = 5
is_irreducible(fx,p)

fx equals to: 1 + 2*x^2 + x^4
which is not irreducible over mod 3 field

p is not a prime number. It's not a filed

fx equals to: 1 + 2*x^2 + x^4
which is not irreducible over mod 5 field

fx equals to: 1 + x^2
which is not irreducible over mod 2 field

fx equals to: 1 + x^2
which is irreducible over mod 3 field

fx equals to: 1 + x^2
which is not irreducible over mod 5 field



## 1.3 Primitive element (20%)

### 1.3.1 Find primitive elements 

Consider the finite field $\mathbb{F}_p$. Design and write a function `find_primitive` to find all primitive elements in $\mathbb{F}_p$. Test your function with a prime number $20 < p < 50$. 

In [8]:
function isprime(n)
    if n < 2
        return false    
    elseif n == 2
        return true
    # Check all even numbers
    elseif n % 2 == 0
        return false
    end

    # Consider all odd numbers
    for i in collect(3:2:√n) # only need to check i < √n
        if n % i == 0 
            return false
        end
    end

    # if no divisors were found
    true  
end

function modExponent(x,n,p)
    sum = x
    while n!=1
      # mult by x again, check if result can be modded
      sum = (sum * x) % p
      n = n-1
    end
    return sum
end

function find_primitive(p)
    if !isprime(p) # Check if it is a prime number
        return println("p is not a prime")
    end
    primElements = Int[]
    for i in 1:p - 1
        order = 1;
        r = 0;
        
        while r != 1 # if r = 1, stop loop
            order = order + 1;
            r = modExponent(i, order, p);
        end

        if order == p - 1 # order equals to p - 1, it is a primitive element
            push!(primElements,i)        
        end
    end
    return primElements
end

testInts = 20:50

# Final string output
outputString = ""

function addToOutput(p,primitives)
    global outputString
    outputString = string(outputString, string("The primitive elements of F<sub>",p,"</sub> are: ", primitives,"<br />"))
end

# Test all ints between 20 and 50 
testInts .|> (x) -> isprime(x) ?  addToOutput(x,find_primitive(x)) : ""

# Display the result as markdown for pretty printing
display("text/markdown",outputString)

The primitive elements of F<sub>23</sub> are: [5, 7, 10, 11, 14, 15, 17, 19, 20, 21]<br />The primitive elements of F<sub>29</sub> are: [2, 3, 8, 10, 11, 14, 15, 18, 19, 21, 26, 27]<br />The primitive elements of F<sub>31</sub> are: [3, 11, 12, 13, 17, 21, 22, 24]<br />The primitive elements of F<sub>37</sub> are: [2, 5, 13, 15, 17, 18, 19, 20, 22, 24, 32, 35]<br />The primitive elements of F<sub>41</sub> are: [6, 7, 11, 12, 13, 15, 17, 19, 22, 24, 26, 28, 29, 30, 34, 35]<br />The primitive elements of F<sub>43</sub> are: [3, 5, 12, 18, 19, 20, 26, 28, 29, 30, 33, 34]<br />The primitive elements of F<sub>47</sub> are: [5, 10, 11, 13, 15, 19, 20, 22, 23, 26, 29, 30, 31, 33, 35, 38, 39, 40, 41, 43, 44, 45]<br />

### 1.3.2 Calculate primitive elements

Now consider the finite field $\mathbb{F}_{32}$. Without programming, can you find all the primitive elements in $\mathbb{F}_{32}$? If so, explain your approach and its rationality. (No programming is needed.)

$\mathbb{F}_{32}$ contains 31 non-0 elements. With $\mathbb{F}_{q}$=$\mathbb{F}_{32}$, we have q=32, and q-1=31 which is a prime number.
$\newline$
According to the corollary of Lemma 2 in the existence of primitive elements (1.35 in the lectures), we know that:
$\newline$
$$\forall \beta \in \mathbb{F}_{32}^*, ord(\beta) | (q-1),$$
$\newline$
where ${F}_{32}^* = \mathbb{F}_{32}$ \ $\{0\}.$
$\newline$
The case where $\beta=1$ is simple, we can clearly see that it is not a primitive element because 1 raised to any power will be 1: it has order 1 (the corollary of Lemma 2 is satisfied because 1 divides 31). 
$\newline$ 
For other $\beta$, q-1 = 31 which is a prime number, so the only number that can divide 31 is 31. Hence, ord($\beta$) | 31 implies that Ord($\beta$) = 31, $\forall$ non-0 elements of the finite field $\mathbb{F}_{32}$ (excluding $\beta=1$). 
$\newline$ 
We know that a primitive element has order q-1=31, so all non-0 elements of $\mathbb{F}_{32}$ (excluding $\beta=1$) are primitive elements: there are a total of 30 primitive elements in $\mathbb{F}_{32}$.

## Highlight

Please list a couple of highlights of your coursework that may impress your markers.

1. Did not use any external package (e.g. Polynomials, DSP), completed all the functions by ourself.

2. We derived our own algorithm for calculating Bézout's coefficients and implemented it in a compact recursive function. (See 1.1.2, Similar idea is also used in 1.2.2).

3. The input format of the polynomial in 1.2 is very flexible, no need to be stuck on whether there are extra 0 (e.g. [1 2 1 0 0 0]), sizes of fx and gx, all can be calculated.

4. In 1.2, we considered all circumstances, no matter what input you can get the correct result.

5. The code is highly modular, split into many small functions. This makes it easy to swap out any function for another and compare performance. (e.g. our own convolution function, function to demonstrate very clear results).